<a href="https://colab.research.google.com/github/utkar22/CSE508_Winter2023_A2_48/blob/main/IR_A2_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import pickle 
import numpy as np
import pandas as pd
import os
import math

# **Jaccard Coefficient**


In [80]:
mod_path = '/content/drive/MyDrive/IR_Assignments/A2/modules'
ds_path = '/content/drive/MyDrive/IR_Assignments/A2/Processed_Dataset'

In [ ]:

doc_id_to_set = {}
idx = 1
for doc in sorted(os.listdir(ds_path)):
  mset = set()

  doc_id = int(doc[-4:])
  doc_path = ds_path + '/' + doc

  print(doc_id)

  with open(doc_path, 'r') as f:
    text = str(f.readlines()[0])

  tokens = text.split(' ')


  for token in tokens:
    mset.add(token)

  print(len(mset))
  print(len(tokens))
  doc_id_to_set[doc_id] = mset

print(len(doc_id_to_set))  
with open(mod_path+'/doc_id_to_mset.pkl', 'wb') as f:
  pickle.dump(doc_id_to_set, f)



In [7]:
#calculating Jaccard coefficient of each query document wise
def calc_jacc(query_tokens, doc_id_to_set):
  res = []
  res_set = {}
  query_set = set()
  for token in query_tokens: #make the set of the query tokens
    query_set.add(token)

  for doc_id, mset in doc_id_to_set.items(): #calculate the intersection and union of each document set and query set
    un, inter = len(query_set.union(mset)), len(query_set.intersection(mset))
    if un == 0: #edge case when the doc or query do not have any words in them
      res_set[doc_id] = 0
      continue
    
    jacc_coeff = inter/un #calculating the Jaccard Coefficient
    res_set[doc_id] = jacc_coeff
  res_set = {k:v for k,v in sorted(res_set.items(), key = lambda item:item[1], reverse = True)} #sorting in descending order on the basis of the Jaccard Coefficient
  idx = 0
  for doc_id, jacc_coeff in res_set.items(): # make the list of the top 10 documents on the basis of the Jaccard coefficient
    res.append((doc_id, jacc_coeff))
    if idx == 9:
      break
    idx+=1

  return res




In [10]:
#query processing and result printing
def jacc_coeff_queries():
  with open(mod_path+'/doc_id_to_mset.pkl', 'rb') as f:
    doc_id_to_set = pickle.load(f)
  q = int(input('Enter number of queries: '))
  while q>0:
    query = input('Enter query: ')

    query_tokens = query.split(' ')
    res = calc_jacc(query_tokens, doc_id_to_set)

    if res[0][1] == 0.0:
      print('No right document exists')
    else:
      print(res)
    q-=1

jacc_coeff_queries()

Enter number of queries: 3
Enter query: velocity temperature aerodynamics
[(407, 0.06896551724137931), (378, 0.05263157894736842), (549, 0.05263157894736842), (71, 0.05128205128205128), (31, 0.05), (269, 0.04878048780487805), (61, 0.047619047619047616), (154, 0.047619047619047616), (460, 0.0425531914893617), (485, 0.041666666666666664)]
Enter query: complete
[(684, 0.03225806451612903), (1038, 0.025), (1400, 0.022222222222222223), (372, 0.021739130434782608), (1078, 0.019230769230769232), (1135, 0.019230769230769232), (702, 0.018867924528301886), (319, 0.018518518518518517), (55, 0.017543859649122806), (1091, 0.017241379310344827)]
Enter query: entry
[(1348, 0.02702702702702703), (715, 0.022727272727272728), (1345, 0.018867924528301886), (944, 0.018518518518518517), (967, 0.01818181818181818), (275, 0.017543859649122806), (1394, 0.016666666666666666), (1346, 0.01639344262295082), (69, 0.015873015873015872), (982, 0.015873015873015872)]


# **TF IDF**

In [13]:
vocabulary = set()

for doc in os.listdir(ds_path):
  with open(ds_path+f'/{doc}', 'r') as f:
    text = f.readlines()[0]
  tokens = text.split(' ')
  for token in tokens:
    vocabulary.add(token)
print(len(vocabulary))

8971


In [84]:
idx = 0
word_to_index = {}
index_to_word = {}
for word in vocabulary:
  word_to_index[word] = idx
  index_to_word[idx] = word
  idx+=1
print(word_to_index)
print(index_to_word)
print(len(word_to_index))
print(len(index_to_word))

{'': 0, 'operators': 1, 'strictly': 2, 'heatsink': 3, 'desk': 4, 'belief': 5, 'tangentwedge': 6, 'contoured': 7, 'pictures': 8, 'structure': 9, 'copper': 10, 'curvedsurface': 11, '37': 12, 'redesigned': 13, 'drum': 14, 'mechanism': 15, 'tabulations': 16, 'interferometer': 17, '0825': 18, 'hemispherecylinder': 19, 'properly': 20, 'nonaxisymmetric': 21, 'october': 22, 'strain': 23, 'laboratories': 24, 'clausing': 25, 'integrodifferential': 26, 'inadequacy': 27, 'family': 28, '1percent': 29, 'bends': 30, 'considers': 31, 'physics': 32, 'spontaneously': 33, 'comprise': 34, '076power': 35, 'glass': 36, 'resolved': 37, 'y0': 38, 'seas': 39, 'tetrachloride': 40, 'rarefied': 41, 'told': 42, 'mostly': 43, '120': 44, '117': 45, 'beam': 46, 'normalshock': 47, 'loadings': 48, 'straindisplacement': 49, 'appraisement': 50, 'exterior': 51, 'approximations': 52, 'blown': 53, '13': 54, 'obeying': 55, 'ablating': 56, 'effusion': 57, 'characteristies': 58, 'supplies': 59, 'acquisition': 60, 'puzzled': 61

In [85]:
with open(mod_path + '/vocabulary_set.pkl', 'wb') as f:
  pickle.dump(vocabulary, f)
with open(mod_path + '/word_to_index.pkl', 'wb') as f:
  pickle.dump(word_to_index, f)
with open(mod_path + '/index_to_word.pkl', 'wb') as f:
  pickle.dump(index_to_word, f)


In [83]:
with open(mod_path + '/vocabulary_set.pkl', 'rb') as f:
  vocabulary = pickle.load(f)

In [37]:
class Word:
  def __init__(self):
    self.binary_tf = {}
    self.count_tf = {}
    self.freq_tf = {}
    self.log_tf = {}
    self.double_norm_tf ={}
    self.df = 0
  

In [69]:
def update_word(token, doc_id, total_words):
  ''' update the binary-tf, count-tf and 
  freq-tf weights of each word per doc'''
  if doc_id not in token.binary_tf:

    token.binary_tf[doc_id] = 1
    token.count_tf[doc_id] = 1

    token.freq_tf[doc_id] = float(token.count_tf[doc_id]/total_words)
    token.log_tf[doc_id] = math.log(1+token.count_tf[doc_id])
    
  else:
    token.count_tf[doc_id] = token.count_tf[doc_id]+1
    token.freq_tf[doc_id] = float(token.count_tf[doc_id]/total_words)

    token.log_tf[doc_id] = math.log(1+token.count_tf[doc_id])
  token.df = len(token.binary_tf)

def update_double_normal_tf(token, doc_id, max_count):
  ''' updates the double normalization tf value for a word'''
  token.double_norm_tf[doc_id] = 0.5 + 0.5*(token.count_tf[doc_id])/max_count


In [ ]:
lst = []
word_set = {}
index = 0
for doc in os.listdir(ds_path):
  local_dict = {}
  doc_idx = int(doc[-4:])
  lst.append(doc_idx)
  with open(ds_path + f'/{doc}', 'r') as f:
    text = f.readlines()[0]
  tokens = text.split(' ')
  print(tokens)
  for token in tokens:
    if token in local_dict:
      local_dict[token] += 1
    else:
      local_dict[token] = 1
    if token not in word_set:
      new_tok = Word()
      word_set[token] = new_tok
    update_word(word_set[token], doc_idx, len(tokens))

  sorted_dict = {k:v for k,v in sorted(local_dict.items(), key = lambda item:item[1], reverse = True)}
  maxi = sorted_dict[list(sorted_dict.keys())[0]]
  for token in tokens:
    update_double_normal_tf(word_set[token], doc_idx, maxi)
  
  # if index == 10:
  #   break
  # index +=1
  # print(doc_idx)

with open(mod_path + f'/word_set.pkl', 'wb') as f:
  pickle.dump(word_set, f)

  # print(sorted(word_set))

In [78]:
print(len(word_set))

8971


In [87]:
with open(mod_path+ '/word_set.pkl', 'rb') as f:
  word_set = pickle.load(f)
with open(mod_path + '/word_to_index.pkl', 'rb') as f:
  word_to_index = pickle.load(f)
with open(mod_path + '/index_to_word.pkl', 'rb') as f:
  index_to_word = pickle.load(f)
print(len(word_set))
print(len(word_to_index))
print(len(index_to_word))


8971
8971
8971


In [118]:
def make_binary_tf_idf_matrix(word_set, V, dir_len):
  binary_mat = np.zeros((dir_len+1, V))

  for token, word in word_set.items():
    map = word.binary_tf
    col = word_to_index[token]
    for row in map.keys():
      binary_mat[row][col] = math.log((1+dir_len)/(1+word.df))

  return binary_mat

binary_mat = make_binary_tf_idf_matrix(word_set, 8971, 1400)
print(np.count_nonzero(binary_mat==0))

12476867


In [133]:
def make_count_tf_idf_matrix(word_set, V, dir_len):
  count_mat = np.zeros((dir_len+1, V))

  for token, word in word_set.items():
    map = word.count_tf
    col = word_to_index[token]
    for row in map.keys():
      count_mat[row][col] = map[row] * math.log((1+dir_len)/(1+word.df))

  return count_mat 

count_mat = make_count_tf_idf_matrix(word_set, 8971, 1400) 
print(np.count_nonzero(count_mat==0))

12476867


In [117]:
def make_freq_tf_idf_matrix(word_set, V, dir_len):
  freq_mat = np.zeros((dir_len+1, V))

  for token, word in word_set.items():
    map = word.freq_tf
    col = word_to_index[token]
    for row in map.keys():
      freq_mat[row][col] = map[row] * math.log((1+dir_len)/(1+word.df))

  return freq_mat 
freq_mat = make_freq_tf_idf_matrix(word_set, 8971, 1400)
print(np.count_nonzero(freq_mat == 0))

12476867


In [120]:
def make_log_tf_idf_matrix(word_set, V, dir_len):
  log_mat = np.zeros((dir_len+1, V))

  for token, word in word_set.items():
    map = word.log_tf
    col = word_to_index[token]
    for row in map.keys():
      log_mat[row][col] = map[row] * math.log((1+dir_len)/(1+word.df))

  return log_mat 
log_mat = make_log_tf_idf_matrix(word_set, 8971, 1400)
print(np.count_nonzero(log_mat == 0))
print(np.unique(log_mat))

12476867
[ 0.          0.49394171  0.5901114  ... 14.73825387 15.08606404
 16.59879738]


In [121]:
def make_dnorm_tf_idf_matrix(word_set, V, dir_len):
  dnorm_mat = np.zeros((dir_len+1, V))

  for token, word in word_set.items():
    map = word.double_norm_tf
    col = word_to_index[token]
    for row in map.keys():
      dnorm_mat[row][col] = map[row] * math.log((1+dir_len)/(1+word.df))

  return dnorm_mat
dnorm_mat = make_dnorm_tf_idf_matrix(word_set, 8971, 1400)
print(np.count_nonzero(dnorm_mat == 0))
print(np.unique(dnorm_mat))

12476867
[0.         0.37505645 0.38175389 ... 6.08380905 6.14632926 6.55179437]


In [122]:
with open(mod_path+'/dnorm_mat.pkl', 'wb') as f:
  pickle.dump(dnorm_mat, f)
with open(mod_path+'/log_mat.pkl', 'wb') as f:
  pickle.dump(log_mat, f)
with open(mod_path + '/freq_mat.pkl', 'wb') as f:
  pickle.dump(freq_mat, f)
with open(mod_path + '/count_mat.pkl', 'wb') as f:
  pickle.dump(count_mat, f)
with open(mod_path + '/binary_mat.pkl', 'wb') as f:
  pickle.dump(binary_mat, f)

In [134]:
# def do_normalization(matrix):
#   n, m =  matrix.shape[0], matrix.shape[1]
#   res_mat = np.zeros((n,m), dtype= np.longdouble)
#   norm_vector = np.linalg.norm(matrix, axis = 1)
#   for row in range(n):
#     for col in range(m):
#       res_mat[row][col] = matrix[row][col]/norm_vector[row]
#   return res_mat
# print(np.unique(count_mat))
# count_mat1 = do_normalization(count_mat)
# print(np.unique(count_mat1))

[ 0.          0.71260725  0.85135079 ... 71.76321015 84.2398594
 93.73835496]


<ipython-input-134-3546f816c230>:7: RuntimeWarning: invalid value encountered in double_scalars
  res_mat[row][col] = matrix[row][col]/norm_vector[row]


[0.         0.00620748 0.00659731 ... 0.84211518 1.                nan]


In [ ]:
#INCOMPLETE RN
def vectorize(query):
  tokens = query.split(' ')
  query_set = {}
  res = {}
  for token in tokens:
    if token in query_set:
      query_set[token] = query_set[token]+1
    else:
      query_set[token] = 1
  sorted_dict = {k:v for k,v in sorted(query_set.items(), key = lambda item:item[1], reverse = True)}
  maxi = sorted_dict[list(sorted_dict.keys())[0]]
  idx = 0
  for idx in range(5):

    for word in query_set.keys():
    
  
    


In [ ]:
def run_tf_idf():
  q = int(input('Enter the number of queries'))
  while q>0:
    query = input('Enter the query')
    tokens = query.split(' ')
    